# **COVID-19 Data Study Notebook**

## Objectives

* Fetch Data from Kaggle and save as raw file and unzip.
* Inspect the data and save it under inputs/datasets/raw
* Inspect the data and save it under outputs/datasets/collection

## Inputs

* Kaggle JSON file - the authentication token.

## Outputs

* Generate Dataset: outputs/datasets/collection/trimmed_covid_dataset.csv

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/milestone-covid-19-study/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/milestone-covid-19-study'

---

# Load Data

In [5]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/covid-19-dataset.csv")
    )
df.head(10)

,USMER,MEDICAL_UNIT,SEX,DATE_DIED,INTUBED,PNEUMONIA,AGE,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,ICU
0,2,1,2,09/06/2020,1.0,2.0,55.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,1,1,9999-99-99,2.0,1.0,40.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2,1,1,9999-99-99,2.0,2.0,37.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
3,2,1,1,9999-99-99,2.0,2.0,25.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2,1,2,9999-99-99,2.0,2.0,24.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,2,1,2,9999-99-99,2.0,2.0,30.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,2,1,1,9999-99-99,2.0,1.0,80.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0
7,2,1,2,9999-99-99,2.0,1.0,59.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0
8,2,1,2,9999-99-99,2.0,1.0,45.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,2,1,2,9999-99-99,1.0,1.0,45.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0


# Data Exploration

We are interested to get more familiar with the dataset, check variable type and distribution, missing levels and what these variables mean in a business context

In [6]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)

pandas_report.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Correlaton Study

In [7]:
from feature_engine.encoding import OneHotEncoder
categorical_vars = df.columns[df.dtypes == 'object'].to_list()
encoder = OneHotEncoder(variables=categorical_vars, drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
print(df_ohe.head(3))

/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/encoding/one_hot.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[str(feature) + "_" + str(category)] = np.where(
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/encoding/one_hot.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[str(feature) + "_" + str(category)] = np.where(
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/encoding/one_hot.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usual

(189417, 380)
   USMER  MEDICAL_UNIT  SEX  INTUBED  PNEUMONIA   AGE  DIABETES  COPD  ASTHMA  \
0      2             1    2      1.0        2.0  55.0       1.0   2.0     2.0   
1      2             1    1      2.0        1.0  40.0       2.0   2.0     2.0   
2      2             1    1      2.0        2.0  37.0       1.0   2.0     2.0   

   INMSUPR  ...  DATE_DIED_02/01/2021  DATE_DIED_22/01/2021  \
0      2.0  ...                     0                     0   
1      2.0  ...                     0                     0   
2      2.0  ...                     0                     0   

   DATE_DIED_26/01/2021  DATE_DIED_14/02/2021  DATE_DIED_19/02/2021  \
0                     0                     0                     0   
1                     0                     0                     0   
2                     0                     0                     0   

   DATE_DIED_22/02/2021  DATE_DIED_14/03/2021  DATE_DIED_27/03/2021  \
0                     0                     0       

Calculate correlations with Spearman method

In [8]:
target_variable = 'DATE_DIED'
corr_spearman = df_ohe.corr(method='spearman')[target_variable].sort_values(key=abs, ascending=False)[1:].head(10)
print("Top 10 Spearman Correlations:")
print(corr_spearman)

KeyError: 'DATE_DIED'

Calculate correlations with Pearson method

In [ ]:
corr_pearson = df_ohe.corr(method='pearson')[target_variable].sort_values(key=abs, ascending=False)[1:].head(10)
print("Top 10 Pearson Correlations:")
print(corr_pearson)

Combine top variables from both methods

In [ ]:
top_n = 5
top_vars = set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())
print("Variables to study:")
print(top_vars)

Filter the dataset for the variables to study

In [ ]:
vars_to_study = list(top_vars) + [target_variable]
df_eda = df.filter(vars_to_study)
print(df_eda.head(3))

## Conclusions and Next Steps

The correlations and plots interpretation converge.

- A churned customer typically has a month-to-month contract.
- A churned customer typically has fiber optic.
- A churned customer typically doesn't have tech support.
- A churned customer doesn't have online security.
- A churned customer typically has low tenure levels